In [ ]:
import holodeck as holo
from holodeck import single_sources, utils, plot, detstats
from holodeck.constants import YR

import numpy as np
import healpy as hp
import matplotlib.pyplot as plt

import hasasia.sensitivity as hsen
import hasasia.sim as hsim
import hasasia.skymap as hsky

# Read in saved sam & strain data

Option 1) Make realizations

In [ ]:
# dur = 15 * YR
# nfreqs = 40
# cad = dur/nfreqs/2
# fobs= utils.nyquist_freqs(dur, cad)
# fobs_edges = utils.nyquist_freqs_edges(dur, cad)
# df_orb = np.diff(fobs_edges/2)
# print(len(fobs))
# sam = holo.sam.Semi_Analytic_Model()
# hc_ss, hc_bg = sam.ss_gwb(fobs_edges, realize=3, loudest=10000)
# np.savez('/Users/emigardiner/GWs/holodeck/ecg-notebooks/detstats_functions/npz_arrays/sam02_r3_f40_l10000.npz', 
#          dur=dur, cad=cad, fobs=fobs, df_orb=df_orb, hc_ss=hc_ss, hc_bg=hc_bg)

Option 2) Load existing

In [ ]:
hcfile = np.load('/Users/emigardiner/GWs/holodeck/ecg-notebooks/detstats_functions/npz_arrays/sam02_r3_f40_l10000.npz')
dur = hcfile['dur']
cad = hcfile['cad']
fobs = hcfile['fobs'][:]
df_orb = hcfile['df_orb']
hc_ss = hcfile['hc_ss']
hc_bg = hcfile['hc_bg']
hcfile.close()

shape = hc_ss.shape
F, R, L = shape[0], shape[1], shape[2]
print('F, R, L =', F,R,L)

# Healpy Tutorial

In [ ]:
# Random sky positions
NSIDE = 32
print( "Approximate resolution at NSIDE {} is {:.2} deg".format(
        NSIDE, hp.nside2resol(NSIDE, arcmin=True) / 60))
NPIX = hp.nside2npix(NSIDE)
print(NPIX)

In [ ]:
m = np.arange(NPIX) # value at each pixel
hp.mollview(m, title="Mollview image RING")
hp.graticule()

In [ ]:
# use a vector to represent coordinates
vec = hp.ang2vec(np.pi / 2, np.pi * 3/4)
print(vec)
ipix_disc = hp.query_disc(nside=32, vec=vec, radius=np.radians(10))
print(ipix_disc)

In [ ]:
print(np.pi*32**2)

In [ ]:
m = np.arange(NPIX)
m[ipix_disc] = m.max()
hp.mollview(m, title="Mollview image RING")

# Place Random Single Sources

In [ ]:
m_strain = np.zeros((F,NPIX)) # frequency, pixel positon
# paint the background evenly across
rr=0 # just one realization
avg_pix_strain = hc_bg[:]/ NPIX # (F,R)
m_strain = np.ones((F,NPIX)) * avg_pix_strain[:,rr,np.newaxis]
print('hc_bg=\n',hc_bg[:,rr], '\nnpix=',NPIX, '\nbgpix strain=\n', (m_strain[:,0]))

# choose random pixels to place single sources, 
pix_ss = np.random.randint(0, NPIX-1, size=F*L).reshape(F,L)
print(pix_ss.shape)

# and add the ss_strains to their pixels
for ff in range(F):
    for ll in range(L):
        m_strain[ff,pix_ss[ff,ll]] += hc_ss[ff,rr,ll]

In [ ]:
min_at_freq = np.min(m_strain, axis=1, )
# max_at_freq = 

In [ ]:
# show in skymap
# ff=23
def ff_mollview(hc_ss, pix_ss, m_strain, ff, plot=False, saveloc=None, ax=None):
    # print(hc_ss[ff,rr,:])
    if plot:
        plt.scatter(np.arange(len(hc_ss[ff,rr,:])), pix_ss[ff,:], c=np.log(hc_ss[ff,rr,:]), cmap='viridis', s=0.5)
        plt.colorbar(label='log(hc_ss)')
        plt.xlabel("L'th Loudest")
        plt.ylabel('Pixel')
    if ax is not None:
        plt.axes(ax)
    hp.mollview(m_strain[ff], title='%d Loudest Single Sources at %.2f yr$^{-1}$ (freq bin %d/%d)' % (L, fobs[ff]*YR, ff, F))
    if saveloc is not None:
        fname = saveloc+'hp_ff%02d.png' % ff
        print(fname)
        plt.savefig(fname, dpi=300)
    plt.close()
fig = ff_mollview(hc_ss, pix_ss, m_strain, 23, plot=True)


## Per Frequency Skymaps

In [ ]:
saveloc = '/Users/emigardiner/GWs/holodeck/ecg-notebooks/healpix/2023-05-11_sam02_l10000/'
for ff in range(F):
    fig = ff_mollview(hc_ss, pix_ss, m_strain, ff, saveloc=saveloc)
    # fig.savefig(saveloc+'hp_ff%02d.png' % ff, dpi=300)
    # plt.close(fig)

In [ ]:
fig, axs = plt.subplots(nrows=10, ncols=int(F/10))
for ff in range(F):
    title='$f$= %.2f yr$^{-1}$ (bin %d/%d)' % (fobs[ff]*YR, ff, F)
    row, col = int(ff/4), ff%4
    # print(row,col)
    plt.axes(axs[row,col])
    hp.mollview(m_strain[ff], title=title, hold=True)

fig.suptitle('%d Loudest Single Sources' % L, y=.895)
fig.tight_layout()
fig.set_size_inches(18, 30)
fig.savefig(saveloc+'all_freqs.png', dpi=1000)

# fig

### 20 frequencies, 1 figure
Using Levi's code

In [ ]:
fig, ((ax0, ax1), (ax2, ax3), (ax4, ax5), (ax6, ax7), (ax8, 
      ax9), (ax10, ax11), (ax12, ax13), (ax14, ax15), (ax16, 
      ax17), (ax18, ax19)) = plt.subplots(nrows=10, ncols=2)

ax_list = [ax0, ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8, 
      ax9, ax10, ax11, ax12, ax13, ax14, ax15, ax16, 
      ax17, ax18, ax19]

for i, a in enumerate(ax_list):
    plt.axes(a)
    hp.mollview(m_strain[i], title="skymap_freqbin{}".format(i), hold=True)
fig.set_size_inches(8, 30)

# Spherical Harmonics

freq bin 5 is nice, lets use that first

In [ ]:
Cl_lmax8 = np.zeros((F,9))
# E_l_lmax8 = np.zeros((F,9))

In [ ]:
for ff in range(F):
    # power spectrum of a healpix map
    Cl_lmax8[ff,:] = hp.anafast(m_strain[ff], lmax=8)
print(Cl_lmax8.shape)

Cl_C0_ratio = Cl_lmax8[:,:]/Cl_lmax8[:,0][:,np.newaxis]
print(Cl_C0_ratio.shape)

l_vals = np.arange(len(Cl_lmax8[0]))

plot all the spherical harmonics

In [ ]:
fig, axs = plt.subplots(nrows=10, ncols=int(F/10), sharex=True, sharey=True)
for ff in range(F):
    title='$f$= %.2f yr$^{-1}$ (bin %d/%d)' % (fobs[ff]*YR, ff, F)
    row, col = int(ff/4), ff%4
    # print(row,col)
    ax = axs[row,col]
    ax.plot(l_vals, Cl_C0_ratio[ff])
    if row==9:
        ax.set_xlabel('$l$')
    if col==0:
        ax.set_ylabel('$C_l/C_0$')

# fig.suptitle('%d Loudest Single Sources' % L, y=.895)
# fig.tight_layout()
fig.set_size_inches(18, 30)
# fig.savefig(saveloc+'all_freqs.png', dpi=1000)

# fig

In [ ]:
import matplotlib.cm as cm
colors=cm.rainbow(np.linspace(0,1,F))
fobs_nHz = fobs*10**9
print(fobs_nHz)

Plot Just Cl

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))

for ff in range(F):
    if ff in (0, 10, 20, 30, 39):
        label= ('$f$ = %.2f nHz' % fobs_nHz[ff])
    else: label = None
    ax.plot(l_vals[:], Cl_lmax8[ff,:], c=colors[ff], alpha=0.5, label=label,
            marker='o')
ax.set_xlabel('$l$')
ax.set_ylabel('$C_l$')
ax.set_yscale('log')
ax.set_title('Spherical Harmonic Coefficients')
ax.legend()
# fig.suptitle('%d Loudest Single Sources' % L, y=.895)
fig.tight_layout()
# fig.savefig(saveloc+'all_freqs.png', dpi=1000)

# fig

### plot C_l/C_0

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))

for ff in range(F):
    if ff in (0, 10, 20, 30, 39):
        label= ('$f$ = %.2f nHz' % fobs_nHz[ff])
    else: label = None
    ax.plot(l_vals[1:], Cl_C0_ratio[ff,1:], c=colors[ff], alpha=0.5, label=label,
            marker='o')
ax.set_xlabel('$\ell$')
ax.set_ylabel('$C_\ell/C_0$')
ax.set_yscale('log')
ax.set_title('Spherical Harmonic Coefficients')
ax.legend()
# fig.suptitle('%d Loudest Single Sources' % L, y=.895)
fig.tight_layout()
# fig.savefig(saveloc+'all_freqs.png', dpi=1000)

# fig

### Plot l(l+1)C_l

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))
xx = l_vals[1:]
yy = (l_vals*(l_vals+1))[np.newaxis,1:]*Cl_lmax8[:,1:]

for ff in range(F):
    if ff in (0, 10, 20, 30, 39):
        label= ('$f$ = %.2f nHz' % fobs_nHz[ff])
    else: label = None
    ax.plot(xx, yy[ff], c=colors[ff], alpha=0.5, label=label,
            marker='o')
ax.set_xlabel('$\ell$')
ax.set_ylabel('$\ell(\ell+1)C_\ell$')
ax.set_yscale('log')
ax.set_title('(Not Normalized) Spherical Harmonics')
ax.legend()
# fig.suptitle('%d Loudest Single Sources' % L, y=.895)
fig.tight_layout()
# fig.savefig(saveloc+'all_freqs.png', dpi=1000)

# fig

## normalizing

In [ ]:
# normalizing each frequency strains by total strain of each frequency
hc2_tot = np.sum(m_strain**2, axis=1)
print(hc2_tot.shape)
print(m_strain.shape)
m_strain_norm = m_strain/np.sqrt(hc2_tot[:,np.newaxis])

Cl_norm=np.zeros((F,9))
for ff in range(F):
    # power spectrum of a healpix map
    Cl_norm[ff,:] = hp.anafast(m_strain_norm[ff], lmax=8)

Cl_C0_ratio_norm = Cl_norm/Cl_norm[:,0,np.newaxis]

#### Plot l(l+1)C_l

In [ ]:
fig, ax = plt.subplots(figsize=(7,5))
xx = l_vals[1:]
yy = (l_vals*(l_vals+1))[np.newaxis,1:]*Cl_norm[:,1:]

for ff in range(F):
    if ff in (0, 10, 20, 30, 39):
        label= ('$f$ = %.2f nHz' % fobs_nHz[ff])
    else: label = None
    ax.plot(xx, yy[ff], c=colors[ff], alpha=0.5, label=label,
            marker='o')
ax.set_xlabel('$\ell$')
ax.set_ylabel('$\ell(\ell+1)C_\ell$')
ax.set_yscale('log')
ax.set_title('Normalized Spherical Harmonics')
ax.legend(loc='lower right')
# fig.suptitle('%d Loudest Single Sources' % L, y=.895)
fig.tight_layout()
# fig.savefig(saveloc+'all_freqs.png', dpi=1000)

# fig